In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.testing_stuff.base_testing import backtest_df_only_testing
from quantfreedom.testing_stuff.enums_testing import (
    CandleBody,
    LeverageMode,
    OrderSettingsArrays,
    OrderType,
    SizeType,
    StaticVariables,
)


np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )

# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf("data/prices.hd5")
rsi_ind = from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = is_below(
    want_to_evaluate=rsi_ind,
    user_args=[40, 50],
    candle_ohlc="close",
)

In [4]:
static_vars = StaticVariables(
    equity=1000.,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    order_type=OrderType.LongEntry,
    size_type=SizeType.RiskPercentOfAccount,
)
order_settings = OrderSettingsArrays(
    max_equity_risk_pct=6.,
    risk_reward=5,
    size_pct=1.,
    sl_based_on=CandleBody.low,
    sl_based_on_lookback=30,
    sl_based_on_add_pct=.2,
)
asdf = backtest_df_only_testing(
    price_data=price_data,
    entries=entries,
    static_variables_tuple=static_vars,
    order_settings_arrays_tuple=order_settings,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 1
Total order settings to test: 5
Total candles per symbol: 9,408
Total candles to test: 188,160

Total combinations to test: 20


RejectedOrderError: Long Increase - Size Value is either to big or too small